# Digging around the Baseball Databank

The Baseball Databank, along with other relevant open baseball datasets, are described and linked to <a href="http://chadwick-bureau.com/open-data/">here</a>, with the current version contained in the <a href="https://github.com/chadwickbureau/baseballdatabank">baseballdatabank git repository</a>. The easiest way to get the data on your machine locally is to clone or download the repository.  Once you have downloaded the data, you will find it contains two folders, 'core' and 'upstream'. The 'core' folder contains the databank itself, and 'upstream' contains files used to construct the databank. 

First, some preliminary imports and configuration

In [ ]:
import pandas as pd
import numpy as np
import glob, os
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

%matplotlib inline

## Reading in BaseballDatabank csv files

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testpydata1/dataframes_baseball)__

First, let's examine the contents of the baseballdatabank/core directory using the %ls magic function in ipython/jupyter.

In [ ]:
%ls data/baseballdatabank/core/

Now, let's define a function that will read all the baseballdatabank csv files into pandas dataframes.

In [ ]:
def read_all_databank_core_csv(directory):
    """
    read all csv files in the specified baseball databank directory and
    populate a dictionary storing each of the tables keyed to its name
    """
    dfs = {}
    files = glob.glob('{}/*.csv'.format(directory))
    for f in files:
        d, name = os.path.split(f)
        table = os.path.splitext(name)[0]
        df = pd.read_csv(f)
        dfs[table] = df
    return dfs

bbdfs = read_all_databank_core_csv('data/baseballdatabank/core')

# let's extract a few for further processing
batting = bbdfs['Batting']
pitching = bbdfs['Pitching']
teams = bbdfs['Teams']

## Taking a peek
### batting is year-by-year for each individual player


In [ ]:
batting.head()

### teams is year-by-year for each team

In [ ]:
teams.head()

In [ ]:
batting.info()

In [ ]:
teams.info()

## Aggregate operations

NumPy and Pandas both provide aggregate mathematical operations that can be carried out over arrays and dataframes, respectively.

With the baseball batting dataframe, we might be interested in the total number of different types of hitting outcomes over the entire history of MLB baseball. We can easily compute this by summing all the rows (axis=0) of the batting dataframe.

In [ ]:
batting.sum(axis=0)

In [ ]:
batting.sum(axis=0, numeric_only=True)

In [ ]:
print(batting['G'].mean())
print(batting['G'].std())

## Adding derived data: calculating singles (1B) from H,2B,3B,HR

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testpydata1/augmenting#derived)__

Sometimes, the dataset that we have access to doesn't contain all the related information that we might want. But we can augment the original dataset with additional data derived from the original. For example, the baseball batting dataset oddly does not contain information about the number of singles (i.e., one-base hits in which the batter reached first base safely and remained there until the next batter batted). In baseball, there are 4 kinds of hits: 
* singles (one base, or 1B)
* doubles (two bases, or 2B)
* triples (three bases, or 3B)
* homeruns (four bases, or HR)

Since the dataset contains information about the total number of hits H, as well as the number of 2B, 3B and HR, we can define an additional column in both the batting and teams dataframes to compute the number of singles (1B). Note the 1B data has been added as the last column.

In [ ]:
batting['1B'] = batting['H'] - batting['2B'] - batting['3B'] - batting['HR']
teams['1B'] = teams['H'] - teams['2B'] - teams['3B'] - teams['HR']
batting.head()

## Exercise: The Slash Line (BA / OBP / SLG)

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testpydata1/augmenting#morederived)__

Having added singles (1B) to the batting dataframe, we can add some more derived data, based upon relationships from the raw data:

* Batting Average (BA) represents the fraction of at bats (AB) that result in a hit of any type.
* On-Base Percentage (OBP) represents roughly the fraction of at bats that result in a batter getting on base through any means (a hit, a walk, or a hit-by-pitch (HBP)). This fraction is corrected somewhat by other factors, and the full formula is OBP = (Hits + Walks + Hit by Pitch) / (At Bats + Walks + Hit by Pitch + Sacrifice Flies).  See __[this link](https://www.baseball-reference.com/bullpen/On_base_percentage)__ for further information.
* Slugging Percentage (SLG) represents the ratio of the average number of bases achieved by a hitter per at-bat (thereby giving greater weight to hits that accrue more bases). 
* In modern day baseball statistics, this triplet of derived data (BA / OBP / SLG) is reported as the "slash line".
* In addition, the composite statistic On-Base-Plus-Slugging (OPS) &mdash; the sum of OBP and SLG &mdash; identifies those highly prized hitters who are both able to get on base effectively and generate extra-base hits, so we might as well add that stat too.
* It should be noted that even though OBP and SLG are referred to as "percentages", they are in fact reported textually as decimal fractions (typically rounded to three decimal places and without a leading integer part if the number is less than 1, e.g., .479).  Perhaps even more confusingly, they are spoken as the integers that would result from multiplying those three-digit decimals by 1000, e.g., a slugging percentage of .800 is spoken as "eight hundred".

<b>Instructions</b>: In the code cell below, augment the <code>batting</code> dataframe by adding four new columns, titled 'BA', 'OBP', 'SLG', and 'OPS', by implementing the appropriate calculation for each.  After doing so, have a look at the dataframe.  If you look at the head of the dataframe (<code>batting.head()</code>), you'll notice that OBP and OPS by these calculations are not well-defined in the early history of baseball, apparently because HBP was not kept track of and is missing from the dataframe.  If you peek at the tail of the dataframe (<code>batting.tail()</code>), you will see these quantities are computed for those players who had at least one AB.  If you're bothered by the missing HBP data, you could try filling in missing data with the <code>batting.fillna()</code> method.

In [ ]:
# TO BE REMOVED
batting['BA']= batting['H'] / batting['AB']
batting['OBP'] = (batting['H']+batting['BB']+batting['HBP']) / (batting['AB']+batting['BB']+batting['HBP']+batting['SF'])
batting['SLG'] = (batting['1B']+2*batting['2B']+3*batting['3B']+4*batting['HR']) / batting['AB']
batting['OPS'] = (batting['OBP']+batting['SLG'])

END of Exercise: The Slash Line

## Groupby operations

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testpydata1/augmenting#groupby)__

An extremely powerful set of capabilities is provided by the <code>groupby</code> method on dataframes, which provides support for <i>split-apply-combine</i> operations.  This means, for example, that we can:
<ul>
    <li><i>split</i> a dataframe into groups based on identity of a specified key or some other criterion
            <li><i>apply</i> an aggregating function across each of the subgroups, and then <li> <i>combine</i> the aggregated information back in a single dataframe.  
    </ul>
Each row in the <code>batting</code> dataframe contains information about a single player in a single year.  What if we wanted to know the totals of all of those statistics, on a year-by-year basis.  We can create a new dataframe by grouping by the <code>'yearID'</code> and then summing each of those year-by-year groups.

In [ ]:
batting_by_year = batting.groupby('yearID').sum().reset_index()
batting_by_year.head()

Alternatively, instead of grouping by year, to get league-wide statistics for each year, we could group by player (playerID) to get career batting statistics for each player.  Alphabetically, the second player on this list (playerID: aaronha01) is Hank Aaron, one of the greatest players of all-time.  (His brother Tommy follows on the list.)

In [ ]:
pl_bat = batting.groupby('playerID').sum().reset_index()
pl_bat.head()

As a technical aside, in both examples above, we chained several operations together in one expression, concluding each with the method <code>reset_index()</code>.  What is that operation for? Recall that the index of a dataframe is the group of labels for each row.  In the raw dataframes that we created when we read in the csv files, the index was simply an incrementing set of integers over each of the rows.  When we execute a groupby operation such as those above, a new dataframe is returned, and the index of that dataframe is the set of labels that we grouped on, e.g., the yearID's in the first example and the playerID's in the second example.  Sometimes it is useful to keep the groupby key as the index, but sometimes you might want to put the key back in as a regular column, and use incrementing integers as an index instead.  That's what the <code>reset_index()</code> method does.

**Note** that the "Slash Line" statistics that we added to the batting dataframe in the exercise above have not been correctly aggregated here in the <code>pl_bat</code> dataframe.  That is because those statistics are all averages of year-by-year counts.  A player's career batting average, however, is not the sum of his year-by-year batting averages (or even the mean of those averages).  Instead, career-wide Slash Line statistics would need to be recomputed based on the aggregate counts in the <code>pl_bat</code> dataframe, as in the code below.  Thus while a dataframe is easily extensible, we also need to keep in mind whether adding new types of data changes the semantics of the table.

In [ ]:
# recompute Slash Line statistics for career batting 
pl_bat['BA'] = pl_bat['H'] / pl_bat['AB']
pl_bat['OBP'] = (pl_bat['H']+pl_bat['BB']+pl_bat['HBP']) / (pl_bat['AB']+pl_bat['BB']+pl_bat['HBP']+pl_bat['SF'])
pl_bat['SLG'] = (pl_bat['1B']+2*pl_bat['2B']+3*pl_bat['3B']+4*pl_bat['HR']) / pl_bat['AB']
pl_bat['OPS'] = (pl_bat['OBP']+pl_bat['SLG'])
pl_bat.head()

## Filtering data
### Top all-time slugging percentages (at least 100 AB)

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testpydata1/filtering_data)__

In [ ]:
pl_bat[pl_bat.AB >= 100].sort_values(by='SLG', ascending=False).head(30)

## A brief visual history of hitting in baseball

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testpydata1/history_baseball)__

We can use visualization tools in seaborn (sns) to summarize the entire history of hitting (batting) in baseball. We are specifically interested here not only in the 4 types of hits, but also some other key batting outcomes: strikeouts (SO) and bases-on-balls (BB), otherwise known as walks, as well as runs scored (R) and runs batting in (RBI).

The dataframe <code>batting_by_year</code> that we created above by computing totals per year is useful for some analyses, but is also confounded by the fact that MLB baseball has grown over time, both in terms of the number of teams in the league and the number of games played in a season.  Thus, per-year totals have grown over time partly due to the simple fact that the number of overall games played per season has increased.  Below we will create a new dataframe that accounts for this effect, by computing batting statistics on a per at-bat (AB) basis.

A PairGrid is used to plot the relationship between all sets of pairs of hitting variables (hit_vars), with which we can overlay with additional color information indicating the year (hue='yearID') to present a succinct history of hitting in baseball. In order to better see the progression of time in this visualization, it is useful to bin the year-by-year data into decades, which we can do by adding a new column to the dataframe (decade) that computes this using the <code>np.floor_divide</code> on each row.  Time progresses from light to dark blue in these plots.

In [ ]:
batting_per_AB_by_year = batting_by_year.copy()
for col in batting_per_AB_by_year.columns:
    if col in ['yearID', 'stint', 'G', 'AB']:
        continue
    batting_per_AB_by_year[col] = batting_per_AB_by_year[col]/batting_per_AB_by_year['AB']

batting_per_AB_by_year['decade'] = np.floor_divide(batting_per_AB_by_year['yearID'],10)*10

hit_vars = ('1B', '2B', '3B', 'HR', 'SO', 'BB', 'R', 'RBI')

pg = sns.pairplot(batting_per_AB_by_year, vars=hit_vars, hue='decade', palette='Blues')
plt.tight_layout()
plt.savefig("images/batting_scatterplot_matrix.png")

## The correlation of hitting statistics

We can summarize the pairwise correlations among hitting outcomes embedded in the scatterplot data above by using the <code>corr()</code> method on the dataframe, and then displaying all the correlation values with a heatmap.

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(batting_per_AB_by_year[list(hit_vars)].corr(), annot=True)
plt.savefig("images/batting_correlation_matrix.png")

## The interoperation of SQLAlchemy and Pandas

__[Link to section in Cornell Virtual Workshop using this material](https://cvw.cac.cornell.edu/testchris/dataframes_sql)__

In [ ]:
def write_all_tables_to_sqlite(dfs, sql_filename):
    engine = create_engine('sqlite:///{}'.format(sql_filename))
    for table, df in dfs.items():
        df.to_sql(table, con=engine, index=False)
    engine.dispose()
    
sqlite_filename = 'bbdb.sqlite'
try:
    os.remove(sqlite_filename)
except FileNotFoundError:
    pass
write_all_tables_to_sqlite(bbdfs, sqlite_filename)

## Making an SQL query to Baseball DB

With all the data in a SQLite database, we can now connect to it and issue SQL queries to extract information of interest.  For example, we can identify which players had the most hits in a season.

In [ ]:
engine = create_engine('sqlite:///bbdb.sqlite')
most_hits_in_a_year = \
    pd.read_sql_query('select * from batting order by H desc limit 10', engine)
most_hits_in_a_year

If you're a baseball fan, maybe you already knew that Ichiro Suzuki (playerID suzukic01) currently holds the record for most hits in a season (262 hits in 2004), having topped George Sisler's long-standing record (257 hits in 1920).  But if you're curious about the names attached to the somewhat obscure playerIDs stored in the batting table, you could issue a more complicated SQL query using a table join to extract the players names from the people table (using Python's triple quotes to create a multiline string to accommodate the long query):

In [ ]:
pd.read_sql_query("""select people.nameFirst, people.nameLast, batting.playerID, batting.yearID, batting.H
from people join batting on people.playerID=batting.playerID order by batting.H desc limit 10""", engine) 

## More baseball history: the saga of home runs

For many baseball fans, the most exciting part of the sport is the home runs.  From our earlier hitting scatterplot, we saw that the rate of home runs has increased steadily over time.  We can examine this in a bit more detail by plotting the rate of HR per AB over time, using the plot method on the batting_per_AB_by_year.  The code block below also includes some annotations relevant to the history of home runs in baseball (and for more information about how baseball was transformed in the 1950s, read https://tht.fangraphs.com/dig_the_1950s/ ). MLB baseball in 2019 is on a pace to obliterate the newly-set 2018 record for most home runs in a season, and many are debating how to try to keep the game from devolving into and endless series of walks, strikeouts and home runs.

In [ ]:
ax = batting_per_AB_by_year.plot('yearID', 'HR', figsize=(16,8))
annot1920 = plt.text(1920, 0.001, '< End of dead ball era')
annot1942 = plt.text(1942, 0.009, '< MLB players in World War II')
annot1950 = plt.text(1950, 0.014, '< 1950s: baseball transformed')
annot1995 = plt.text(1995, 0.020, '< Steroids rampant')
annot2003 = plt.text(2003, 0.0225, '< Steroids tested for')
annot2015 = plt.text(2015, 0.025, '< Fascination with launch angle')
plt.xlabel('year', fontsize='x-large')
plt.ylabel('average number of HR per AB', fontsize='x-large')
plt.tight_layout()
plt.savefig('images/history_of_homeruns.png')